In [8]:
!pip install torchsummary
import os
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from torch.utils.data import Dataset
import pandas as pd
from torch import nn
from torchsummary import summary
from torch.utils.data import DataLoader

In [9]:
class UrbanSoundDataset(Dataset):
    
    def __init__(self, annotations_file, audio_dir, transformations, sample_rate, num_samples, device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformations = transformations.to(self.device)
        self.sample_rate = sample_rate
        self.num_samples = num_samples
        
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self.resample(signal, sr)
        signal = self.mix_down(signal)
        signal = self.cut(signal)
        signal = self.right_pad(signal)
        signal = self.transformations(signal)
        return signal, label
    
    def cut(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal
    
    def right_pad(self, signal):
        length = signal.shape[1]
        if length<self.num_samples:
            num_missing = -length+self.num_samples
            signal = torch.nn.functional.pad(signal, (0,num_missing))
        return signal
    
    def resample(self,signal, sr):
        if sr != self.sample_rate:
            signal = F.resample(signal, sr, self.sample_rate)
        return signal
    
    def mix_down(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal


    
    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[index, 0])
        return path
    
    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]

In [10]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device}")

Using cuda


In [15]:
BATCH_SIZE = 1024
EPOCHS = 30
LEARNING_RATE = 0.0001

ANNOTATIONS_FILE = "/kaggle/input/urbansound8k/UrbanSound8K.csv"
AUDIO_DIR = "/kaggle/input/urbansound8k"
SAMPLE_RATE = 22050
NUM_SAMPLES = 22050

mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE,n_fft=1024,hop_length=512,n_mels=64)

usd = UrbanSoundDataset(ANNOTATIONS_FILE,AUDIO_DIR,mel_spectrogram,SAMPLE_RATE,NUM_SAMPLES,device)

print(usd.__len__())

8732


In [12]:
def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader


def train_single_epoch(model, data_loader, loss_fn, optimiser, device):
    for input, target in data_loader:
        input, target = input.to(device), target.to(device)

        # calculate loss
        prediction = model(input)
        loss = loss_fn(prediction, target)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")

In [13]:
class CNNNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=2,bias=False),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2))

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=2*16,kernel_size=3,stride=1,padding=2,bias=False),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2))

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=4*16,kernel_size=3,stride=1,padding=2,bias=False),
        nn.BatchNorm2d(2*32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2))

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=2*32,out_channels=8*16,kernel_size=3,stride=1,padding=2,bias=False),
        nn.BatchNorm2d(4*32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2))
        
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(128*5*4,10) # 10 classes
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        logits = self.linear(x)
        preds = self.softmax(logits)

        return preds

In [14]:
train_dataloader = create_data_loader(usd, BATCH_SIZE)

# construct model and assign it to device
cnn = CNNNetwork().to(device)
print(cnn)

# initialise loss funtion + optimiser
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(cnn.parameters(),
                             lr=LEARNING_RATE)

# train model
train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

# save model
torch.save(cnn.state_dict(), "feedforwardnet.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1

In [16]:
# train model
train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

# save model
torch.save(cnn.state_dict(), "feedforwardnet.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

Epoch 1
loss: 1.8331711292266846
---------------------------
Epoch 2
loss: 1.8579210042953491
---------------------------
Epoch 3
loss: 1.8244266510009766
---------------------------
Epoch 4
loss: 1.834548830986023
---------------------------
Epoch 5
loss: 1.8366326093673706
---------------------------
Epoch 6
loss: 1.822311282157898
---------------------------
Epoch 7
loss: 1.8178892135620117
---------------------------
Epoch 8
loss: 1.8496674299240112
---------------------------
Epoch 9
loss: 1.8059686422348022
---------------------------
Epoch 10
loss: 1.8102777004241943
---------------------------
Epoch 11
loss: 1.8371118307113647
---------------------------
Epoch 12
loss: 1.7984271049499512
---------------------------
Epoch 13
loss: 1.8216724395751953
---------------------------
Epoch 14
loss: 1.796482801437378
---------------------------
Epoch 15
loss: 1.785979986190796
---------------------------
Epoch 16
loss: 1.7890472412109375
---------------------------
Epoch 17
loss: 1.8232